In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import json

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
'''
Hyperparameters
    Neural network 
        # of hidden layers
        # of hidden layer nodes
    Training
        epochs
        batch_size
        learning_rate
        scheduler(StepLR)
            gamma
            step_size
''' 
no_hidden_layers = 5
no_hidden_nodes = 64
epochs = 2000
batch_size=128
learning_rate = 1e-03
gamma=0.97
step_size=100

In [ ]:
def train_minmax_extraction_current(x,y,eps=1e-12,is_nmos=True):
    '''
    For train data preprocessing
    Caution: Extract this variables, only train data!
    '''
    global vds_min, vds_max, vgs_min, vgs_max, vbs_min, vbs_max, Id_min, Id_max
    x = np.array(x).ravel()
    y = np.array(y).ravel()
    vds_min, vds_max = x[:,0].min(), x[:,0].max()
    vgs_min, vgs_max = x[:,1].min(), x[:,1].max()
    vbs_min, vbs_max = x[:,2].min(), x[:,2].max()
    if is_nmos is False:
        y *= -1
    y = np.where(y<0, 0, y) + eps
    y_dum = np.log10(y) # min value == np.log10(eps)
    assert y_dum.min() == np.log10(eps), "Check your data or preprocess equation"
    Id_min, Id_max = y_dum.min(), y_dum.max()

def train_minmax_extraction_charge(x,y,eps=1e-12,is_nmos=True):
    '''
    For train data preprocessing
    Caution: Extract this variables, only train data!
    '''
    global vds_min, vds_max, vgs_min, vgs_max, vbs_min, vbs_max, Qd_min, Qd_max, Qg_min, Qg_max, Qb_min, Qb_max
    x = np.array(x).ravel()
    y = np.array(y).ravel()
    vds_min, vds_max = x[:,0].min(), x[:,0].max()
    vgs_min, vgs_max = x[:,1].min(), x[:,1].max()
    vbs_min, vbs_max = x[:,2].min(), x[:,2].max()
    Qd_min, Qd_max = y[:,0].min(), y[:,0].max()
    Qg_min, Qg_max = y[:,1].min(), y[:,1].max()
    Qb_min, Qb_max = y[:,2].min(), y[:,2].max()
    
def MyPreprocess_I(x,y, eps=1e-12, is_nmos=True):
    '''
    x -> Vds, Vgs Vbs
    y -> Ids
    Preprocess equation
        X
            Vds' = (x[:,0] - vds_min)/(vds_max - vds_min)
            Vgs' = (x[:,1] - vgs_min)/(vgs_max - vgs_min)
            Vbs' = (x[:,2] - vbs_min)/(vbs_max - vbs_min)
            prec_x = [Vds', Vgs', Vbs']
        Y
            y = cleansing y 
            y_dum = np.log10(y)
            prec_y = (y_dum - Id_min)/(Id_max - Id_min)
            ! cleansing: 
                - y < 0 data elimination 
                - y += eps
    is_nmos:
        default: True(NMOS transistor), False(PMOS transistor)
            -> In calse of False, y *= -1
    '''
    x,y = np.array(x).ravel(), np.array(y).ravel()
    x[:,0] = (x[:,0] - vds_min) / (vds_max - vds_min)
    x[:,1] = (x[:,1] - vgs_min) / (vgs_max - vgs_min)
    x[:,2] = (x[:,2] - vbs_min) / (vbs_max - vbs_min)
    prec_x = x

    if is_nmos is False:
        y *= -1
    y = np.where(y<0, 0, y) + eps
    y_dum = np.log10(y) # min value == np.log10(eps)
    prec_y = (y_dum - Id_min)/(Id_max - Id_min)
    return prec_x, prec_y

def MyPostprocess_I(y,is_nmos=True):
    '''
    Reversal of preprocess procedure
    y_dum = y*(Id_max - Id_min) + Id_min
    post_y = 10**y_dum

    '''
    y_dum = y*(Id_max - Id_min) + Id_min
    post_y = 10**y_dum
    if is_nmos is False:
        post_y *= -1
    return post_y

def MyPreprocess_Q(x,y, eps=1e-12, is_nmos=True):
    '''
    x -> Vds, Vgs Vbs
    y -> Qd, Qg, Qb
    Preprocess equation
        X
            Vds' = (x[:,0] - vds_min)/(vds_max - vds_min)
            Vgs' = (x[:,1] - vgs_min)/(vgs_max - vgs_min)
            Vbs' = (x[:,2] - vbs_min)/(vbs_max - vbs_min)
            prec_x = [Vds', Vgs', Vbs']
        Y
            prec_y = (y_dum - Id_min)/(Id_max - Id_min)
    '''

class MyDataset(Dataset):
    '''
    Custom dataset -> simple MOSFET data preprocessing 
    '''
    def __init__(self,data_numpy,is_nmos=True,is_current=True):
        self.data_numpy = data_numpy
        self.is_nmos = is_nmos
        self.is_current = is_current
        self.x = self.data_numpy[:,:3]
        self.y = self.data_numpy[:,4]
        if self.is_current:
            Mypreprocess = Mypreprocess_I
        else:
            Mypreprocess = Mypreprocess_Q
        self.x, self.y = Mypreprocess(self.x, self.y, self.is_nmos)
    def __getitem__(self, idx):
        return torch.DoubleTensor(self.x[idx,:]).to(device), torch.DoubleTensor(self.y[idx,:]).view(-1,1).to(device)
    def __len__(self):
        return len(self.x[:,0])

In [ ]:
# Train_data, Valid_data, Test_data


train_dataset = MyDataset(train_data)
valid_dataset = MyDataset(valid_data)
test_dataset = MyDataset(test_data)
train_dataloader = DataLoader(train_dataset, batch_size=, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=, shuffle=False)

In [ ]:
class Simple_FC(nn.Module):
    def __init__(self,input_nodes=3,output_nodes=1,num_hidden_layer=5,num_hidden_nodes=64):
        self.input_nodes = input_nodes
        self.output_nodes = output_nodes
        self.num_hidden_layer = num_hidden_layer
        self.num_hidden_nodes = num_hidden_nodes
        self.fc_layers = nn.ModuleList()
        # input_layer
        self.fc_layers.append(nn.Linear(self.input_nodes, self.num_hidden_nodes))
        # hidden_layer
        for _ in range(self.num_hidden_layer):
            self.fc_layers.append(nn.Linear(self.num_hidden_nodes, self.num_hidden_nodes))
        # output_layer
        self.output_layer = nn.Linear(self.num_hidden_nodes,self.output_nodes)
        self.elu = F.elu()

    def forward(self, x):
        for layer in self.fc_layers:
            x = layer(x)
            x = self.elu(x)
        output = self.output_layer(x)
        return output

In [ ]:
# model, optimization, scheduler definition
myModel_I = Simple_FC(input_nodes=3, output_nodes=1, num_hidden_layer=no_hidden_layers, num_hidden_nodes=no_hidden_nodes)
optimizer = optim.Adam(myModel_I.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

In [ ]:
# Training - I model
best_loss = np.inf
cnt = 0
for epoch in range(epochs):
    cnt += 1
    train_loss = 0
    for x, y in train_dataloader:
        y_pred = myModel_I(x)
        loss = torch.mean((y-y_pred)**2)
        optimzer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    scheduler.step()

    valid_loss = 0
    with torch.no_grad():
        for x, y in valid_dataloader:
            y_pred = myModel_I(x)
            loss = torch.mean((y-y_pred)**2)
            valid_loss += loss.item()
        
    if valid_loss < best_loss:
        best_loss = valid_loss
        best_model = copy.deepcopy(myModel)
        best_epoch = epoch

    if epochs // cnt == 10:
        cnt = 0
        print(f'Training log - Progress: {(epoch+1)/epochs*100}%')
        print(f'Training loss: {train_loss}, Validation loss: {valid_loss}, Best validation loss: {best_loss} at {best_epoch}.')

# Test data Quality check

In [17]:
# model save - jit trace method use!
torch.jit.save(best_model,'./BSS_Imodel_NN.pt')

# I model - data parameter save
params = {
    "VDS": {
        "min": vds_min,
        "max": vds_max
    },
    "VGS":{
        "min": vgs_min,
        "max": vgs_max
    },
    "VBS":{
        "min": vbs_min,
        "max": vbs_max
    },
    "Id": {
        "min": Id_min,
        "max": Id_max
    },
}
with open('./BSS_Imodel_params.json','w') as json_file:
    json.dump(params,json_file,indent=4)

10

In [ ]:
# model, optimization, scheduler definition
myModel_Q = Simple_FC(input_nodes=3, output_nodes=3, num_hidden_layer=no_hidden_layers, num_hidden_nodes=no_hidden_nodes)
optimizer = optim.Adam(myModel_Q.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

In [ ]:
# Training - Q model
best_loss = np.inf
cnt = 0
for epoch in range(epochs):
    cnt += 1
    train_loss = 0
    for x, y in train_dataloader:
        y_pred = myModel_Q(x)
        loss = torch.mean((y-y_pred)**2)
        optimzer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    scheduler.step()

    valid_loss = 0
    with torch.no_grad():
        for x, y in valid_dataloader:
            y_pred = myModel_Q(x)
            loss = torch.mean((y-y_pred)**2)
            valid_loss += loss.item()
        
    if valid_loss < best_loss:
        best_loss = valid_loss
        best_model = copy.deepcopy(myModel)
        best_epoch = epoch

    if epochs // cnt == 10:
        cnt = 0
        print(f'Training log - Progress: {(epoch+1)/epochs*100}%')
        print(f'Training loss: {train_loss}, Validation loss: {valid_loss}, Best validation loss: {best_loss} at {best_epoch}.')

# Test data Quality check

In [ ]:
# model save - jit trace method use!
torch.jit.save(best_model,'./BSS_Qmodel_NN.pt')

# Q model - data parameter save
params = {
    "VDS": {
        "min": vds_min,
        "max": vds_max
    },
    "VGS":{
        "min": vgs_min,
        "max": vgs_max
    },
    "VBS":{
        "min": vbs_min,
        "max": vbs_max
    },
    "Qd": {
        "min": Qd_min,
        "max": Qd_max
    },
    "Qg": {
        "min": Qg_min,
        "max": Qg_max
    },
    "Qb": {
        "min": Qb_min,
        "max": Qb_max
    },
}
with open('./BSS_Qmodel_params.json','w') as json_file:
    json.dump(params,json_file,indent=4)